# More infomation of the model
https://spacy.io/models/en#en_core_web_trf


https://huggingface.co/roberta-base

# Before Fine-turning

In [ ]:
!python -m spacy download en_core_web_trf

In [2]:
import spacy
import pandas as pd

In [3]:
nlp = spacy.load("en_core_web_trf")

In [4]:
cs_text = """
Stable Diffusion is a deep learning, text-to-image model released by startup StabilityAI in 2022. 
It is primarily used to generate detailed images conditioned on text descriptions, 
though it can also be applied to other tasks such as inpainting, outpainting, and generating image-to-image translations guided by a text prompt.
"""

In [5]:
output = nlp(cs_text)
for ent in output.ents:
  print(ent.text, ent.label_)

StabilityAI ORG
2022 DATE


# Prepare training data: train.spacy 

prepare sciNER dataset for fineturning

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
train_data_path = "/content/drive/MyDrive/fine-turning-dataset/sciNER/train.json"
df_train = pd.read_json(train_data_path, lines=True)
df_train.head()

,clusters,sentences,ner,relations,doc_key
0,"[[[17, 20], [23, 23]]]","[[English, is, shown, to, be, trans-context-fr...","[[[0, 0, Material], [10, 10, OtherScientificTe...","[[], [[29, 29, 31, 32, CONJUNCTION], [48, 49, ...",J87-1003
1,"[[[62, 64], [90, 91], [96, 98], [112, 114]], [...","[[In, this, paper, ,, a, novel, method, to, le...","[[[6, 6, Method], [10, 12, OtherScientificTerm...","[[[6, 6, 10, 12, USED-FOR], [10, 12, 14, 16, U...",CVPR_2003_18_abs
2,"[[[154, 154], [214, 214]], [[40, 44], [85, 85]...","[[In, this, paper, ,, we, present, a, digital,...","[[[7, 13, Method], [15, 21, Method], [23, 25, ...","[[[7, 13, 15, 21, USED-FOR], [15, 21, 23, 25, ...",INTERSPEECH_2013_31_abs
3,"[[[3, 3], [110, 110]]]","[[We, propose, a, method, that, automatically,...","[[[3, 3, Generic], [7, 7, OtherScientificTerm]...","[[[3, 3, 7, 7, USED-FOR], [7, 7, 20, 23, USED-...",I05-5008
4,"[[[35, 35], [69, 69]]]","[[Graph, unification, remains, the, most, expe...","[[[0, 1, Task], [8, 10, Task]], [[16, 17, Meth...","[[[0, 1, 8, 10, PART-OF]], [[16, 17, 22, 23, P...",C92-2068


In [8]:
print(" ".join(df_train['sentences'][0][0]))
print(df_train['ner'][0][0])

English is shown to be trans-context-free on the basis of coordinations of the respectively type that involve strictly syntactic cross-serial agreement .
[[0, 0, 'Material'], [10, 10, 'OtherScientificTerm'], [17, 20, 'OtherScientificTerm']]


In [9]:
processed_data = []
for index, row in df_train.iterrows():
  abstract = [" ".join(i) for i in row["sentences"]]
  abstract = " ".join(abstract)
  abstract_list = [item for sublist in row["sentences"] for item in sublist]
  
  for num in range(len(row["sentences"])):
    sentence = row["sentences"][num]
    entity_list = row['ner'][num]
    if entity_list is not None and len(entity_list) != 0:
      text = " ".join(sentence)
      updated_entity_list = []
      
      for entity_tuple in entity_list:
        (start, end, label) = entity_tuple
        phrase = " ".join(abstract_list[start:end+1])
        phrase = phrase + " "
        if len(phrase) > 3 and phrase in text:
          
          updated_entity_list.append((text.index(phrase), text.index(phrase) + len(phrase)-1, label))
      
      entity = [tuple(sublist)for sublist in updated_entity_list]
      data_tuple = (text, entity)
      processed_data.append(data_tuple)

print(len(processed_data))

1762


In [10]:
print(processed_data[0])

('English is shown to be trans-context-free on the basis of coordinations of the respectively type that involve strictly syntactic cross-serial agreement .', [(0, 7, 'Material'), (58, 71, 'OtherScientificTerm'), (110, 151, 'OtherScientificTerm')])


In [11]:
text = "English is shown to be trans-context-free on the basis of coordinations of the respectively type that involve strictly syntactic cross-serial agreement ."
print(text[0:7])
print(text[58:71])
print(text[110:151])

English
coordinations
strictly syntactic cross-serial agreement


In [12]:
text = "The agreement in question involves number in nouns and reflexive pronouns and is syntactic rather than semantic in nature because grammatical number in English , like grammatical gender in languages such as French , is partly arbitrary ."
print(text[4:13])
print(text[45:50])
print(text[55:73])
print(text[130:148])
print(text[152:159])
print(text[167:185])
print(text[189:198])
print(text[207:213])

agreement
nouns
reflexive pronouns
grammatical number
English
grammatical gender
languages
French


In [13]:
text = "The formal proof , which makes crucial use of the Interchange Lemma of Ogden et al. , is so constructed as to be valid even if English is presumed to contain grammatical sentences in which respectively operates across a pair of coordinate phrases one of whose members has fewer conjuncts than the other ; it thus goes through whatever the facts may be regarding constructions with unequal numbers of conjuncts in the scope of respectively , whereas other arguments have foundered on this problem ."
print(text[50:67])
print(text[127:134])

Interchange Lemma
English


In [14]:
train_data = []
index = 0
for i in processed_data:
  text, annotation = i
  train_data.append((text, annotation))

print(len(train_data))
print(train_data[0])

1762
('English is shown to be trans-context-free on the basis of coordinations of the respectively type that involve strictly syntactic cross-serial agreement .', [(0, 7, 'Material'), (58, 71, 'OtherScientificTerm'), (110, 151, 'OtherScientificTerm')])


https://dzone.com/articles/how-to-fine-tune-bert-transformer-with-spacy-3

In [15]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en")
db = DocBin()
index = 0
for i in tqdm(train_data):
  index += 1
  for text, annotations in [i]:
      # print(index)
      try:
        doc = nlp(text)
        ents = []
        for start, end, label in annotations:
            span = doc.char_span(start, end, label=label)
            ents.append(span)
        doc.ents = ents
        db.add(doc)
      except:
        continue
  
db.to_disk("/content/train.spacy")

100%|██████████| 1762/1762 [00:00<00:00, 2975.64it/s]


In [16]:
db.from_disk("/content/train.spacy")
len(db)

1623

# Prepare dev data: dev.spacy

In [17]:
dev_data_path = "/content/drive/MyDrive/fine-turning-dataset/sciNER/dev.json"
df_dev = pd.read_json(dev_data_path, lines=True)
df_dev

,clusters,sentences,ner,relations,doc_key
0,"[[[6, 17], [32, 32]], [[4, 4], [55, 55], [91, ...","[[This, paper, presents, an, algorithm, for, c...","[[[4, 4, Generic], [6, 17, Task], [20, 21, Mat...","[[[4, 4, 6, 17, USED-FOR], [20, 21, 4, 4, USED...",ICCV_2003_158_abs
1,"[[[90, 91], [107, 107]]]","[[Past, work, of, generating, referring, expre...","[[[4, 5, OtherScientificTerm], [12, 13, OtherS...","[[], [], [], [], [], []]",C04-1096
2,"[[[32, 32], [44, 44]], [[1, 2], [11, 11]], [[1...","[[An, entity-oriented, approach, to, restricte...","[[[1, 2, Method], [4, 5, Task]], [[11, 11, Gen...","[[[1, 2, 4, 5, USED-FOR]], [], [[32, 32, 37, 3...",P84-1047
3,"[[[6, 11], [21, 21], [53, 53]], [[15, 16], [69...","[[This, paper, summarizes, the, formalism, of,...","[[[4, 11, Task], [6, 11, OtherScientificTerm],...","[[[15, 16, 19, 19, USED-FOR]], [], [[61, 62, 5...",C88-1066
4,"[[[34, 36], [99, 101]], [[3, 5], [27, 27], [48...","[[We, present, a, text, mining, method, for, f...","[[[3, 5, Method], [8, 9, OtherScientificTerm],...","[[[3, 5, 8, 9, USED-FOR], [13, 14, 3, 5, USED-...",C04-1116
5,"[[[28, 31], [68, 70], [96, 96], [123, 123]], [...","[[In, this, work, ,, we, present, a, technique...","[[[7, 7, Generic], [9, 10, Task], [17, 22, Oth...","[[[7, 7, 9, 10, USED-FOR], [7, 7, 28, 31, USED...",ICCV_2009_47_abs
6,"[[[58, 59], [71, 72], [94, 95]], [[8, 10], [22...","[[An, attempt, has, been, made, to, use, an, A...","[[[8, 10, Method], [14, 15, Method]], [[22, 22...","[[[8, 10, 14, 15, HYPONYM-OF]], [[40, 41, 33, ...",C80-1073
7,"[[[4, 6], [25, 25], [65, 65], [70, 70], [88, 8...","[[We, present, a, practically, unsupervised, l...","[[[4, 6, Method], [9, 10, OtherScientificTerm]...","[[[4, 6, 9, 10, USED-FOR], [15, 17, 20, 22, US...",H05-1041
8,[],"[[We, revisit, the, classical, decision-theore...","[[[3, 9, Task], [12, 14, Method]], [[32, 34, O...","[[[12, 14, 3, 9, USED-FOR]], [], [[50, 52, 55,...",NIPS_2014_18_abs
9,"[[[51, 57], [74, 74]], [[7, 8], [70, 71]], [[3...","[[We, analyze, a, reweighted, version, of, the...","[[[3, 8, Method], [7, 8, Method], [12, 18, Tas...","[[[3, 8, 12, 18, USED-FOR], [12, 18, 22, 23, F...",NIPS_2014_10_abs


In [18]:
processed_data = []
for index, row in df_dev.iterrows():
  # if index == 39:
  abstract = [" ".join(i) for i in row["sentences"]]
  abstract = " ".join(abstract)
  abstract_list = [item for sublist in row["sentences"] for item in sublist]
  
  for num in range(len(row["sentences"])):
    sentence = row["sentences"][num]
    entity_list = row['ner'][num]
    if entity_list is not None and len(entity_list) != 0:
      text = " ".join(sentence)
      updated_entity_list = []
      for entity_tuple in entity_list:
        (start, end, label) = entity_tuple
        phrase = " ".join(abstract_list[start:end+1])
        phrase = phrase + " "
        if len(phrase) > 3 and phrase in text:
          
          updated_entity_list.append((text.index(phrase), text.index(phrase) + len(phrase)-1, label))
      
      entity = [tuple(sublist)for sublist in updated_entity_list]
      data_tuple = (text, entity)
      processed_data.append(data_tuple)

print(len(processed_data))

260


In [19]:
dev_data = []
index = 0
for i in processed_data:
  text, annotation = i
  dev_data.append((text, annotation))

print(len(dev_data))

260


In [21]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en")
db = DocBin()
index = 0
for i in tqdm(dev_data):
  index += 1
  for text, annotations in [i]:
      try:
        doc = nlp(text)
        ents = []
        for start, end, label in annotations:
            span = doc.char_span(start, end, label=label)
            ents.append(span)
        doc.ents = ents
        db.add(doc)
      except:
        continue

db.to_disk("/content/dev.spacy")

100%|██████████| 260/260 [00:00<00:00, 1560.15it/s]


In [22]:
import spacy
from spacy.tokens import DocBin
db = DocBin()
db.from_disk("/content/dev.spacy")
len(db)

241

# fine-turning

In [ ]:
!pip install scispacy
!pip install spacy-transformers

In [24]:
!python -m spacy init fill-config /content/base_config.cfg /content/config_spacy.cfg

✔ Auto-filled config with all values
✔ Saved config
/content/config_spacy.cfg
You can now add your data and train your pipeline:
python -m spacy train config_spacy.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [25]:
!python -m spacy train /content/config_spacy.cfg --output /content/output --paths.train /content/train.spacy --paths.dev /content/dev.spacy --gpu-id 0

✔ Created output directory: /content/output
ℹ Saving to output directory: /content/output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-11-24 18:18:22,491] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-11-24 18:18:22,502] [INFO] Pipeline: ['transformer', 'ner']
INFO:spacy:Pipeline: ['transformer', 'ner']
[2022-11-24 18:18:22,502] [INFO] Resuming training for: ['ner']
INFO:spacy:Resuming training for: ['ner']
[2022-11-24 18:18:22,507] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-11-24 18:18:22,508] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from

In [27]:
import spacy

nlp = spacy.load("/content/drive/MyDrive/fine-turning-dataset/sciNER_fineturn/roBERTa/output/model-best")
cs_text = """
Stable Diffusion is a deep learning, text-to-image model released by startup StabilityAI in 2022. 
It is primarily used to generate detailed images conditioned on text descriptions, 
though it can also be applied to other tasks such as inpainting, outpainting, and generating image-to-image translations guided by a text prompt.
"""
output = nlp(cs_text)
for ent in output.ents:
  print(ent.text, ent.label_)

Stable Diffusion Method
deep learning, text-to-image model Method
StabilityAI Method
text descriptions OtherScientificTerm
tasks Generic
inpainting Task
outpainting Task
generating image-to-image translations Task
text prompt OtherScientificTerm


In [ ]:
!zip -r /content/fine_turned_output.zip /content/output

In [ ]:
from google.colab import files
files.download("/content/fine_turned_output.zip")